# Load data

Sätze extrahiert aus der UN-Generaldebatte 2022

In [ ]:
!test -f sentences.txt || wget https://github.com/datanizing/m3-llm-workshop/raw/main/sentences.txt

In [ ]:
sentences = open("sentences.txt").read().split("@@@")

# Encode sentences

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/e5-base-v2')

In [ ]:
# can take a minute or two depending on CPU/GPU configuration
sembeddings = model.encode(["passage: " + s for s in sentences], show_progress_bar=True)

In [ ]:
import numpy as np
with open("sentences-e5.npy", "wb") as f:
    np.save(f, sembeddings)

In [ ]:
sembeddings.shape

In [ ]:
model2 = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# can take a minute or two depending on CPU/GPU configuration
sembeddings2 = model2.encode(sentences, show_progress_bar=True)

In [ ]:
import numpy as np
with open("sentences-mpnet.npy", "wb") as f:
    np.save(f, sembeddings2)

# Retrieval

In [ ]:
def search(query, text, corpus_embeddings, model, top=20):
    # code query to restrict search space
    question_embedding = model.encode(query)
    
    # Determine similarity (vectors are normalized)
    sim = np.dot(corpus_embeddings, question_embedding)
    
    # Get most similar top by sorting
    hits = [ { "text": text[i], "score": sim[i] } 
                     for i in sim.argsort()[::-1][0:top] ]
    
    # Return as dataframe
    return pd.DataFrame(hits)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [ ]:
search("query: Is the climate crisis worse for poorer countries?", 
       sentences, sembeddings, model)

In [ ]:
search("query: Is the war on Ukraine caused by Russia?", 
       sentences, sembeddings, model)

In [ ]:
search("query: Is the climate crisis worse for poorer countries?", 
       sentences, sembeddings2, model2)

In [ ]:
search("Is the war on Ukraine caused by Russia?", 
       sentences, sembeddings2, model2)